In [1]:
from llama_index.core import Document, VectorStoreIndex, Settings, StorageContext
import json
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core.llms import ChatMessage
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore

In [9]:
llm = Ollama(model="llama3.2:3b", request_timeout=120.0)
Settings.llm = llm
embedding_model = OllamaEmbedding(
    model_name="mxbai-embed-large",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0},
)
Settings.embed_model = embedding_model

In [3]:
with open("../data/processed/resolutions_processed.json") as f:
    data = json.load(f)

In [4]:
documents = [
    Document(
        text=item["full_text"],
        metadata={
            "name": item["name"],
            "resolution_date": item["resolution_date"],
            "concept": item["concept"],
            "process_date": item["process_date"],
        },
    )
    for item in data
]

In [5]:
db = chromadb.PersistentClient(path="../../../chroma_db")
chroma_collection = db.get_or_create_collection("creg_index")

In [6]:
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [10]:
index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    async_insert=True,
    embedding_model=embedding_model
)

In [11]:
query_engine = index.as_query_engine()
response = query_engine.query("Dime sobre la RESOLUCIÓN No. 502 073 DE 2024 de la CREG")
response

Response(response='Esta resolución se emitió el 11 de julio de 2024 y aborda la nulidad de la Resolución CREG 502 013 de 2023, solicitada por una empresa. La Comisión de Regulación de Energía y Gas (CREG) estudió todos los argumentos allegados por los solicitantes y basó su decisión en factores objetivamente relevantes. En consecuencia, la CREG decidió no reponer la mencionada Resolución 502 013 de 2023. La resolución también establece que se notificará a varias entidades y se publicará en el Diario Oficial.', source_nodes=[NodeWithScore(node=TextNode(id_='c4f370e5-9544-45ee-81bd-fc7557c55d98', embedding=None, metadata={'name': 'RESOLUCIÓN No. 502 073 DE 2024', 'resolution_date': '2024-07-11', 'concept': '', 'process_date': '2025-01-19'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='a306ee13-6d73-4e78-aa03-fee8726b761c', node_type='4', metadata={'name': 'RESOLUCIÓN No. 502 073 DE 2024', 'resoluti

In [12]:
response.response

'Esta resolución se emitió el 11 de julio de 2024 y aborda la nulidad de la Resolución CREG 502 013 de 2023, solicitada por una empresa. La Comisión de Regulación de Energía y Gas (CREG) estudió todos los argumentos allegados por los solicitantes y basó su decisión en factores objetivamente relevantes. En consecuencia, la CREG decidió no reponer la mencionada Resolución 502 013 de 2023. La resolución también establece que se notificará a varias entidades y se publicará en el Diario Oficial.'